In [4]:
#!/usr/bin/env python
# coding: utf-8

# In[89]:


# Load the Pandas libraries with alias 'pd' 
import pandas as pd 
import numpy as np
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
data = pd.read_csv("train.csv") 
# Preview the first 5 lines of the loaded data 
data.head()


# In[90]:


data.at[4,'number_project']
data.iat[4,2]


# In[91]:


len(data.columns)


# In[92]:


msk = np.random.rand(len(data)) < 0.8
train = data[msk]
val = data[~msk]
print(len(train))
print(len(val))
train.head()


# In[93]:


val.head()


# In[179]:


#train = train.loc[:,['Work_accident', 'promotion_last_5years', 'department', 'salary','left'] ] 
print(train.head())
#val = val.loc[:,['Work_accident', 'promotion_last_5years', 'department', 'salary','left'] ] 
print(val.head())


# In[95]:


#class DecisionTreeForDTClassifier:
#   def __init__(self,)
depAttValue = list(data["time_spend_company"].unique())
depAttValue.sort()
print(depAttValue)
print(len(depAttValue))
print(data.columns)


9009
2229
   satisfaction_level  last_evaluation  number_project  average_montly_hours  \
0                0.10             0.90               7                   286   
1                0.89             0.93               4                   249   
4                0.84             0.84               5                   163   
7                0.85             0.49               4                   139   
9                0.66             0.85               5                   271   

   time_spend_company  Work_accident  left  promotion_last_5years  \
0                   4              0     1                      0   
1                   3              0     0                      0   
4                   3              0     0                      0   
7                   2              0     0                      0   
9                   4              0     0                      0   

         sales  salary  
0        sales     low  
1        sales     low  
4    technical     

In [14]:

# In[186]:


#Implement the solution using scikit learn algorithm
numberOfNodes = 0


# In[96]:


class Node(object):
    def __init__(self, label,name,parentAttrValue,nodeNumber,level):
        self.name = name
        self.label = label
        self.parentAttrValue = parentAttrValue 
        self.nodeNumber = nodeNumber
        self.level = level
        self.children = []

    def add_child(self, obj):
        self.children.append(obj)
        
    def __str__(self):
        return self.label +"  "+ self.name+"  "+self.parentAttrValue+"Node number is "+ str(self.nodeNumber)+" of level "+str(self.level)


targetAttribute = 'left'
Attributes = list(train.columns)
Attributes.remove(targetAttribute)
AllAttrExpTar = Attributes[:]
print(Attributes)
examples = list(train.index.values) 
len(examples)

def getEntropyOfExamples(examplesForCurAttr):
    NumberOfPosEx = 0
    NumberOfNegEx = 0
    for j in examplesForCurAttr:
        if  train.at[int(j),targetAttribute] == 1:
            NumberOfPosEx = NumberOfPosEx+1
        elif train.at[int(j),targetAttribute] == 0:
             NumberOfNegEx = NumberOfNegEx+1
    PropOfPos =  NumberOfPosEx/len(examplesForCurAttr)
    PropOfNeg = NumberOfNegEx/len(examplesForCurAttr)
    return ((-1*PropOfPos*(np.log(PropOfPos)/np.log(2))) + (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2))))
    

def getEntropyofAttribute(currentAttribute,examplesForCurAttr,targetAttribute):
    NumberOfexamplesForCurAttr = len(examplesForCurAttr)
    sum = 0.0
    fra1 = train.loc[examplesForCurAttr,[currentAttribute] ]
    currentAttributeValues = fra1[currentAttribute].unique()
    #currentAttributeList = fra1[currentAttribute]
    #unique2, counts2 = np.unique(currentAttributesList, return_counts=True)
    #dictOfAttr = dict(zip(unique2, counts2))
    #print("currentAttribute is "+currentAttribute+" and its values are")
    #print(currentAttributeValues)
    for i in currentAttributeValues:
    #for i in unique2:
        NumberOfExaForThisValue = 0
        for j in examplesForCurAttr:
            if train.at[int(j),currentAttribute] == i:
                NumberOfExaForThisValue = NumberOfExaForThisValue+1
        if NumberOfExaForThisValue == 0:
            sum = sum+0
        else:
            NumberOfPosForThisValue = 0
            NumberOfNegForThisValue = 0
            for j in examplesForCurAttr:
                if  train.at[int(j),targetAttribute] == 1 and train.at[int(j),currentAttribute] == i:
                    NumberOfPosForThisValue = NumberOfPosForThisValue+1
                elif train.at[int(j),targetAttribute] == 0 and train.at[int(j),currentAttribute] == i:
                    NumberOfNegForThisValue = NumberOfNegForThisValue+1
            PropOfPos = NumberOfPosForThisValue/NumberOfExaForThisValue
            PropOfNeg = NumberOfNegForThisValue/NumberOfExaForThisValue
            PropOfThisValue = NumberOfExaForThisValue/NumberOfexamplesForCurAttr
            #print("PropOfPos "+str(PropOfPos)+" PropOfNeg "+str(PropOfNeg)+"NumberOfexamplesForCurAttr "+str(NumberOfexamplesForCurAttr))
            sum = sum + ((PropOfThisValue)*((-1*PropOfPos*(np.log(PropOfPos)/np.log(2))) + (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2)))))
    return sum

def BestOfAttributes(Attributes,examplesForCurAttr,targetAttribute,method):
    if len(Attributes) == 1:
        return Attributes[0]
    elif method == 'Entropy':
        EntropyForAttributes = {}
        for i in Attributes:
            EntropyForAttributes[i] = getEntropyofAttribute(i,examplesForCurAttr,targetAttribute)
        #findining the minimum among entropy of attributes
        #print("entropy of attributes")
        #print(EntropyForAttributes)
        BestOfAttributes = min(EntropyForAttributes, key=EntropyForAttributes.get)
        return BestOfAttributes        
    elif method == 'Gini':
        pass
    elif method == 'misClasifi':
        pass
    return BestOfAttributes

def DecisionTreeBuilder(examplesForCurAttr,targetAttribute,Attributes,parentAttrValue,level):
    # if all examplesForCurAttr are positive(i.e employee left the company,left =1 )
    global numberOfNodes
    #level
    allExPos = True
    for i in examplesForCurAttr:
        if train.at[i,targetAttribute] == 1:
            pass
        else:
            allExPos = False
            break
    if allExPos:
        r = Node('Positive','leaf Node',parentAttrValue,numberOfNodes,level)
        numberOfNodes  = numberOfNodes + 1
        print("********************************************************************")
        print("The label of current node is")
        print(r)
        #print("number of examplesForCurAttr of this attribute is ")
        #print(len(examplesForCurAttr))
        #print(examplesForCurAttr)
        print("Type1")
        print("********************************************************************")    
        return r
    
    # if all examplesForCurAttr are negative(i.e employee didnot leave the company,left =0 )
    allExNeg = True
    for j in examplesForCurAttr:
        if (train.at[j,targetAttribute] == 0):
            pass
        else:
            allExNeg = False
            break
    if allExNeg:
        r = Node('Negative','leaf Node',parentAttrValue,numberOfNodes,level)
        numberOfNodes  = numberOfNodes + 1
        print("********************************************************************")
        print("The label of current node is")
        print(r)
        #print("number of examplesForCurAttr of this attribute is ")
        #print(len(examplesForCurAttr))
        #print(examplesForCurAttr)
        print("Type2")
        print("********************************************************************")
        return r
    
    #if attributes are empty , then most common value of target Attribute in examplesForCurAttr
    if len(Attributes) == 0 or len(examplesForCurAttr)<50 or getEntropyOfExamples(examplesForCurAttr) < 0.3:
        print("entropy of current examples ")
        print(str(getEntropyOfExamples(examplesForCurAttr)))
        fra1 = train.loc[examplesForCurAttr,[targetAttribute] ]
        targetAttributesList = fra1[targetAttribute].values
        unique1, counts1 = np.unique(targetAttributesList, return_counts=True)
        dictOfTaAttr = dict(zip(unique1, counts1))
        posCount = dictOfTaAttr[1]
        negCount = dictOfTaAttr[0]
        if posCount>negCount:
            r = Node('Positive','leaf Node',parentAttrValue,numberOfNodes,level)
            numberOfNodes  = numberOfNodes + 1
        else:
            r = Node('Negative','leaf Node',parentAttrValue,numberOfNodes,level)
            numberOfNodes  = numberOfNodes + 1
        print("********************************************************************")
        print("The label of current node is")
        print(r)
        #print("number of examplesForCurAttr of this attribute is ")
        #print(len(examplesForCurAttr))
        #print(examplesForCurAttr)
        #print("PosCount "+str(posCount)+"and "+str(negCount)+" negCount")
        print("Type3")
        print("********************************************************************")
        return r
    #if len(Attributes) == 0:
    #else:
        #Attributes = Attributes + AllAttrExpTar
    
    BestAttribute = BestOfAttributes(Attributes,examplesForCurAttr,targetAttribute,'Entropy')
    Attributes.remove(BestAttribute)    
    r = Node(BestAttribute,'internalNode',parentAttrValue,numberOfNodes,level)
    numberOfNodes  = numberOfNodes + 1
    BestAttributeValues = list(train[BestAttribute].unique())
    for iter in BestAttributeValues:
        AttributesForNextNodes = []
        AttributesForNextNode = Attributes[:]
        examplesForCurAttrForThisChild = []
        for iter1 in examplesForCurAttr:
            if  train.at[int(iter1),str(BestAttribute)] == iter:
                examplesForCurAttrForThisChild.append(int(iter1)) 
        tempstr = str(iter)+str(BestAttribute)
        r.add_child(DecisionTreeBuilder(examplesForCurAttrForThisChild,targetAttribute,AttributesForNextNode,tempstr,level+1))
    print("********************************************************************")
    print("The label of current node is")
    print(r)
    print("the best decision attribute of current node is")
    print(BestAttribute)
    #print("number of examplesForCurAttr of this attribute is ")
    #print(len(examplesForCurAttr))
    #print(examplesForCurAttr)
    #print("children of current nodes")
    plabo = range(0,len(r.children))
    for k in plabo:
        pass
        #print(r.children[k])
    print("Type4")
    print("********************************************************************")
            
    return r


# In[194]:


RootNode = DecisionTreeBuilder(examples,'left',Attributes,'root',0) 
numberOfNodes



['Work_accident', 'promotion_last_5years', 'department', 'salary']
entropy of current examples 
0.9541008103738108
********************************************************************
The label of current node is
Negative  leaf Node  0promotion_last_5yearsNode number is 4 of level 4
Type3
********************************************************************
entropy of current examples 
0.5032583347756457
********************************************************************
The label of current node is
Negative  leaf Node  1promotion_last_5yearsNode number is 5 of level 4
Type3
********************************************************************
********************************************************************
The label of current node is
promotion_last_5years  internalNode  salesdepartmentNode number is 3 of level 3
the best decision attribute of current node is
promotion_last_5years
Type4
********************************************************************
entropy of current examples

entropy of current examples 
0.41381685030363374
********************************************************************
The label of current node is
Negative  leaf Node  0promotion_last_5yearsNode number is 35 of level 4
Type3
********************************************************************
entropy of current examples 
1.0
********************************************************************
The label of current node is
Negative  leaf Node  1promotion_last_5yearsNode number is 36 of level 4
Type3
********************************************************************
********************************************************************
The label of current node is
promotion_last_5years  internalNode  salesdepartmentNode number is 34 of level 3
the best decision attribute of current node is
promotion_last_5years
Type4
********************************************************************
entropy of current examples 
0.2351933818192415
*********************************************************

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: RuntimeWarning: divide by zero encountered in log
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: RuntimeWarning: invalid value encountered in double_scalars


entropy of current examples 
0.8555444410875038
********************************************************************
The label of current node is
Negative  leaf Node  0promotion_last_5yearsNode number is 65 of level 4
Type3
********************************************************************
********************************************************************
The label of current node is
Negative  leaf Node  1promotion_last_5yearsNode number is 66 of level 4
Type2
********************************************************************
********************************************************************
The label of current node is
promotion_last_5years  internalNode  0Work_accidentNode number is 64 of level 3
the best decision attribute of current node is
promotion_last_5years
Type4
********************************************************************
entropy of current examples 
0.41381685030363374
********************************************************************
The label of current n

entropy of current examples 
0.7300166301457939
********************************************************************
The label of current node is
Negative  leaf Node  0promotion_last_5yearsNode number is 97 of level 4
Type3
********************************************************************
entropy of current examples 
0.5435644431995964
********************************************************************
The label of current node is
Negative  leaf Node  1promotion_last_5yearsNode number is 98 of level 4
Type3
********************************************************************
********************************************************************
The label of current node is
promotion_last_5years  internalNode  0Work_accidentNode number is 96 of level 3
the best decision attribute of current node is
promotion_last_5years
Type4
********************************************************************
entropy of current examples 
0.42080948947668984
*******************************************

122

In [15]:
Attributes

['Work_accident', 'promotion_last_5years', 'department']